# 第3回レポート課題

学籍番号：XX-YYY
氏名：ZZZZ ZZZZ

CIFAR10という画像データセットに対して、50000枚の学習用画像を用いて独自のニューラルネットワークを学習し、学習したニューラルネットワークを10000枚のテスト用画像に適用して、その結果を報告せよ。<font color="red">特に、下記の点に従って、本レポートを作成すること。
    
- 下記に指定されたフォーマットで、ニューラルネットワークの学習・テスト用コードを実装すること


- 下記に指定されたフォーマットで各処理を数学的に記述すること

</font>

なお、CIFAR10の詳細情報に関しては、下記の公式ページを参照するか、もしくはその他多くのWebページを調べれば分かる。

https://www.cs.toronto.edu/~kriz/cifar.html

## 1. データの読み込みと説明

### 下記の要領で、4つのnumpyデータを読み込め。（5点）

- `train_data.npy`：50000枚の学習用画像が格納されている本データを読み込み、`x_train`という変数に格納せよ。各画像は、サイズが縦32ピクセル、横32ピクセルで、3チャンネルのRGBデータである。<font color="red">特に、`x_train`は、行数が50000、列数が3072の行列（numpy array）とする。</font>行数は学習用画像の数で明らかであるが、列数は「RGBの3つの値で表された32$\times$32個のピクセル」を **1行に整形**することによる（つまり、$3072 = 32 \times 32 \times 3$）。


- `train_labels.npy`：50000枚の学習用画像のラベルが格納されいる本データを読み込み、`y_train`という変数に格納せよ。<font color="red">特に、`y_train`は、行数が50000、列数が1の行列（numpy array）とする。</font>つまり、各行が、各学習用画像が属するクラスのIDを表している。


- `test_data.npy`：10000枚のテスト用画像が格納されている本データを読み込み、`x_test`という変数に格納せよ。学習用画像と同様、各画像は、サイズが32x32で、3チャンネルのRGBデータである。<font color="red">上述の`x_train`と同様の仕様に従い、`x_test`は、行数が10000、列数が3072の行列（numpy array）とする。</font>


- `test_labels.npy`：10000枚のテスト用画像のラベルが格納されている本データを読み込み、`y_test`という変数に格納せよ。<font color="red">上述の`y_train`と同様の仕様に従い、`y_test`は、行数が10000、列数が1の行列（numpy array）とする。</font>


<font color="blue">「どの画像が学習用で、どの画像がテスト用か」という学習用画像とテスト用画像の分割は、絶対に変更しないこと！ 変更した場合は、大幅に減点する。</font>また、データを読み込んだ後で、以降のニューラルネットワークの学習、テストで必要な**データの正規化**を行ってもらって構わない。この正規化は採点外である。

In [9]:
# コードを記述するセル
import numpy as np

# np.reshapeで50000x3072のnumpy arrayに変更.
x_train = np.reshape(np.load('train_data.npy'), (50000, 3072))

# np.reshapeで50000x1のnumpy arrayに変更.
y_train = np.reshape(np.load('train_labels.npy'), (50000, 1))

# np.reshapeで10000x3072のnumpy arrayに変更.
x_test = np.reshape(np.load('test_data.npy'), (10000, 3072))

# np.reshapeで10000x1のnumpy arrayに変更.
y_test = np.reshape(np.load('test_labels.npy'), (10000, 1))

# 以下正規化
x_train = x_train.astype('float32') # float32型に変換
x_test = x_test.astype('float32')
x_train /= 255 # RGB値を[0,255]から[0.0,1.0へ]
x_test /= 255

import keras

# 今回の場合ではIDが10個あるので, 1の場合なら[1,0,0,0,0,0,0,0,0,0]
# 2なら[0,1,0,0,0,0,0,0,0,0]という風に表している.
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# 以下, 確認用
# print(x_train)
# print(y_train)

### 上記で読み込んだデータのうち、$P=50000$個の学習用画像を$\left\{ (\mathbf{x}_p, y_p) \right\}_{p=1}^{P}$としたとき、$\mathbf{x}_p$と$y_p$が、何を表しているか**数学的に**説明せよ。（5点）

つまり、

- $\mathbf{x}_p$は、$p$番目の学習用画像の特徴をどのように表現しているベクトルか


- $y_p$は、$p$番目の学習用画像のラベルをどのように表現しているか

について説明せよ。詳細な説明は不要で、$\mathbf{x}_p$、$y_p$をそれぞれ、1, 2文で**簡潔かつ直感的に**説明することが望ましい。

$\mathbf{x}_p$は$p$番目の学習用画像の色の並びを1行で表しており, $y_p$は$p$番目の学習用画像のラベルを1行で表している.

## 2. 学習するモデルの実装と説明

### `x_train`と`y_train`を用いて学習する複数の層からなるニューラルネットワークを実装せよ。（5点）

層数、ユニット数、活性化関数などのニューラルネットワークの構造は、各自の任意とする。また、自分でニューラルネットワークを実装するのではなく、kerasというディープラーニングライブラリを用いることをお勧めする。kerasのインストール方法に関しては、下記を参照するとよい。

https://www.info.kindai.ac.jp/~shirahama/courses/ml/2018/slides/installation_guide_mac.pdf

もちろん、自分で実装したり、keras以外の別のライブラリを使用してもらっても構わない。

In [10]:
from sklearn.model_selection import train_test_split # ②
from sklearn.neural_network import MLPClassifier # ③ ④ ⑤
from sklearn.metrics import accuracy_score, \
    classification_report, confusion_matrix # ⑥
import pickle  # (A)
import numpy as np
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import Normalizer 



### 上記で実装したニューラルネットワークを**数学的に**説明せよ。（5点）

つまり、それぞれの層の各ユニットで、どのような計算が行われているのか式で示せ。ただし、以下の点に留意すること。

- <font color="red">1層目のユニットの入力は、$\mathbf{x}_p$とする。</font>


- 1層目のユニットの入力以外は、自由に記号を定義してもらって構わない。ただし、定義した記号は、必ず説明すること。


- 式で記述する層は、入力層、基底関数の層（全結合層）、活性化関数の層、出力層だけでよい。授業では解説していないDropout、バッチ正規化などの層を用いる場合は、それらの層の概念的な説明のみでよい。


- 例えば、同じような基底関数の層を複数使用したとすると、同じような計算を繰り返して書く必要が出てくる。その場合は、例えば「この層のユニットの計算はX層目と同様」というように省略してもらって構わない。

# 入力層

# 基底関数の層

# 活性化関数の層

# 出力層

## 3. 学習プロセスの実装と説明

### `x_train`、`y_train`を用いて、上記で定義したニューラルネットワークを学習するコードを実装せよ。（5点）

kerasなどのディープラーニングライブラリを用いれば、数行で書けると思われる。もちろん、自分で学習用のコードを実装しても構わないが、かなり長いコードになるはずなのでお勧めはしない。

また、学習の設定（例えば、オプティマイザー、エポック数、バッチサイズなど）をコメントとして記述すること。

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 512)               1573376   
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 10)               

### 上記のニューラルネットワークの学習で用いたコスト関数（損失関数）を数学的に説明せよ。（5点）

以下の点に留意すること。

- <font color="red">上で定義した学習例の集合$\left\{ (\mathbf{x}_p, y_p) \right\}_{p=1}^{P}$を用いること。</font>


- 説明対象は、最上位層におけるコスト関数である。つまり、コスト関数は、$p$番目の学習用画像の特徴$\mathbf{x}_p$に対するニューラルネットワークの出力（**自分で定義する**）と、$p$番目の学習用画像のラベル$y_p$を比較するはずである。

（数学的説明を記述するセル）

## 4. テストプロセスの実装と説明

### `x_test`と`y_test`を用いて学習したニューラルネットワークをテストするコードを実装せよ。（5点）

特に、テスト結果として<font color="red">精度（Accuracy）</font>を出力せよ。

Test loss: 1.2700451587677002
Test accuracy: 0.5467000007629395


### 上記のテストにおける精度の計算方法を数学的に説明せよ。（5点）

以下の点に留意すること。

- <font color="red">Q=10000個のテスト用画像$\left\{ (\mathbf{x}_q, y_q) \right\}_{q=1}^{Q}$とすること
    
    
- $q$番目のテスト用画像に対する学習済みのニューラルネットワークの出力（**自分で定義する**)と、$q$番目のテスト用画像のラベル$y_q$を比較するはずである。学習済みのニューラルネットワークの出力は、上記の学習プロセスで定義したものとよく似てくるはずである。

（数学的説明を記述するセル）

## <font color="red">特典：精度が高い上位5名は、10点加点する！</font>